# Batch Result Download
Download a batch of Conveyal regional analysis results. This script was streamlined in April 2024 with the v7.2 Conveyal / R5 release.

Setup requires:
 - Completed regional analysis results, viewable in the Conveyal user interface
 - JSON with array values for the key `analysisIds` saved to `config/regionalAnalysisParameters.json`. The values in the array can be derived from the URL of regional analysis results pages in the Conveyal user interface: `https://analysis.conveyal.com/regions/[regionId]/regional/[analysisId]`
 - A current Conveyal token (e.g. 'bearer 1234abcd...') saved at `config/.auth`, based on the `idToken` shown at https://analysis.conveyal.com/session (for logged in users).

After the setup cell in this notebook, the remaining cell: 
 - Downloads a .zip file with all combinations of analyses, cutoffs, destination pointsets, and percentiles

## Setup

In [ ]:
import requests
import json
import urllib
import os
import time

config = json.load(open('config/regionalAnalysisParameters.json'))

# Authorization header with idToken copied from https://analysis.conveyal.com/session
token = open('config/.auth').readline().strip()
headers = {
    'Authorization': token
}
baseUrl = 'https://api.conveyal.com/api/regional/'

## Download regional analysis result geotiff files
For all analysis, cutoff, percentile, and destination combinations

In [ ]:
# Loop over analysis ids
for analysisId in config['analysisIds']:
    
    print('Processing ' + analysisId)
    status = 202;
    
    while (status == 202):
        resultUrl = baseUrl + analysisId + '/all'
        # Request a signed S3 url from the Conveyal API
        r = requests.get(resultUrl, headers = headers, verify = False)
        if r.status_code == 403:
            print('Unauthorized access. Your authorization token may be invalid or expired.')
            
        elif r.status_code == 404:
            print('Results not found. Check your analysisId values')
        
        elif r.status_code == 202:
            print(r.text + ' Retrying in 15 seconds.')
            time.sleep(15)
        
        elif (r.status_code != 200):
            print('Error: ' + r.text)
        
        status = r.status_code
            
    if (status == 200):
        zipRequest = requests.get(r.json()['url'], verify = False)

        # Save result to a local .zip file
        with open('results/'  + analysisId + '.zip', 'wb') as f:
            for chunk in zipRequest.iter_content(chunk_size=128):
                f.write(chunk)
            f.close()